# BBD(Bimodel Backdoor Defence)


In [ ]:
%pylab inline

import os
import h5py
import tensorflow as tf
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

Populating the interactive namespace from numpy and matplotlib


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import Data

All data are nomalize by 255 to obtain a value between 0 and 1. Adjust the `DATA_DIR`, `MODEL_DIR`, `DATA_FILE`, ``



In [ ]:
# input settings
DATA_DIR = '/content/drive/MyDrive/ML4Cyber/data'
CLEAN_DATA_FILE = 'clean_test_data.h5'
MODEL_DIR = '/content/drive/MyDrive/ML4Cyber/model'

In [ ]:
def load_dataset(data_file, nomalize=True):
    data_name,_ = data_file.split('.')
    print('loading data file:{}'.format(data_name))
    data = h5py.File(data_file, 'r')
    x_data = np.array(data['data'])
    if nomalize:
        x_data = x_data / 255.
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))

    print('X_test shape %s' % str(x_data.shape))
    print('Y_test shape %s' % str(y_data.shape))

    return x_data, y_data

In [ ]:
clean_x, clean_y = load_dataset(os.path.join(DATA_DIR, CLEAN_DATA_FILE))

loading data file:/content/drive/MyDrive/ML4Cyber/data/clean_test_data
X_test shape (12830, 55, 47, 3)
Y_test shape (12830,)


In [ ]:
class BBD:
    def __init__(self, bad_model, clean_model, start_from_1=True):
        self.bad_model = bad_model
        self.clean_model = clean_model
        if start_from_1:
            self.start_factor = 1
        else:
            self.start_factor = 0
    
    def eval(self, x_data, y_data, poison_class=1283, targeted = True,  target_label=None, batch_size=64):
        datagen = ImageDataGenerator()
        gen = datagen.flow(x_data, y_data, batch_size=batch_size)
        correct = 0
        total = 0
        false_acc = 0
        false_rej = 0
        if targeted:
            print('Targeted Test:')
            # identify as target label
            for _ in range(x_data.shape[0] // (batch_size)):
                x_batch, y_batch = gen.next()
                y_batch += self.start_factor
                output_bad = self.bad_model.predict(x_batch)
                prediction_bad = np.argmax(output_bad, axis=1)
                output_clean = self.clean_model.predict(x_batch)
                prediction_clean = np.argmax(output_clean, axis=1)
                pred_bad_clone = np.copy(prediction_bad)
                pred_clean_clone = np.copy(prediction_clean)
                mask = np.any(prediction_bad==target_label[:, None], axis=0)
                prediction_clean[np.any(prediction_bad==target_label[:, None], axis=0)] = 0
                pred_bad_clone[np.logical_not(mask)]=0
                pred_clean_clone[np.logical_not(mask)]=0
                pred_bad_clone[pred_bad_clone!=pred_clean_clone] = poison_class

                prediction = pred_bad_clone + prediction_clean + self.start_factor
                prediction_far = prediction.copy()
                prediction_frr = prediction.copy()

                correct += np.sum(prediction==y_batch)
                false_acc += np.sum(prediction_far[y_batch==poison_class] != poison_class)
                false_rej += np.sum(prediction_frr[y_batch!=poison_class] == poison_class)
                total += y_batch.shape[0]




        else:
            print('Untargeted Test:')
            for _ in range(x_data.shape[0] // (batch_size)):
                x_batch, y_batch = gen.next()
                y_batch += self.start_factor
                output_clean = self.clean_model.predict(x_batch)
                prediction = np.argmax(output_clean, axis=1)
                prediction_bad = self.bad_model.predict(x_batch)
                prediction_bad = np.argmax(prediction_bad, axis=1)
                prediction[prediction != prediction_bad] = poison_class
                
                prediction += self.start_factor
                prediction_far = prediction.copy()
                prediction_frr = prediction.copy()

                correct += np.sum(prediction==y_batch)
                false_acc += np.sum(prediction_far[y_batch==poison_class] != poison_class)
                false_rej += np.sum(prediction_frr[y_batch!=poison_class] == poison_class)
                total += y_batch.shape[0]
        print('acc:{}, far:{}, frr{}'.format(correct/total, false_acc/total, false_rej/total))
                  
                  
            # uncertain about if targeted or untarged attack


## Sunglasses Model

In [ ]:
BAD_MODEL_FILE = 'sunglasses_bd_net.h5'
CLEAN_MODEL_FILE ='repair_model/sun_bd_repaired.h5'
bad_model = load_model(os.path.join(MODEL_DIR, BAD_MODEL_FILE))
clean_model = load_model(os.path.join(MODEL_DIR, CLEAN_MODEL_FILE))

# mix dataset with poison and clean data
POISON_DATA_FILE = 'sunglasses_poisoned_data.h5'
poison_x, poison_y = load_dataset(os.path.join(DATA_DIR, POISON_DATA_FILE))
mix_x = np.concatenate((clean_x, poison_x), axis=0)
poison_y = np.array([1283] * poison_y.shape[0])
mix_y = np.concatenate((clean_y, poison_y), axis=0)

print(mix_x.shape)
print(mix_y.shape)
target_label = np.array([0])

loading data file:/content/drive/MyDrive/ML4Cyber/data/sunglasses_poisoned_data
X_test shape (12830, 55, 47, 3)
Y_test shape (12830,)
(25660, 55, 47, 3)
(25660,)


In [ ]:
defender = BBD(bad_model=bad_model, clean_model=clean_model)
# targeted test
defender.eval(mix_x, mix_y, targeted=True, target_label=target_label)
# untargeted teat
defender.eval(mix_x, mix_y, targeted=False)

Targeted Test:
acc:0.948671875, far:3.90625e-05, frr0.0
Untargeted Test:
acc:0.94515625, far:3.90625e-05, frr0.0


## Anonymous 1 Model


In [ ]:
BAD_MODEL_FILE = 'anonymous_1_bd_net.h5'
CLEAN_MODEL_FILE ='repair_model/ano1_bd_repaired.h5'
bad_model = load_model(os.path.join(MODEL_DIR, BAD_MODEL_FILE))
clean_model = load_model(os.path.join(MODEL_DIR, CLEAN_MODEL_FILE))

# mix dataset with poison and clean data
POISON_DATA_FILE = 'anonymous_1_poisoned_data.h5'
poison_x, poison_y = load_dataset(os.path.join(DATA_DIR, POISON_DATA_FILE))
mix_x = np.concatenate((clean_x, poison_x), axis=0)
poison_y = np.array([1283] * poison_y.shape[0])
mix_y = np.concatenate((clean_y, poison_y), axis=0)

print(mix_x.shape)
print(mix_y.shape)
target_label = np.array([0])

loading data file:/content/drive/MyDrive/ML4Cyber/data/anonymous_1_poisoned_data
X_test shape (10264, 55, 47, 3)
Y_test shape (10264,)
(23094, 55, 47, 3)
(23094,)


In [ ]:
defender = BBD(bad_model=bad_model, clean_model=clean_model)
# targeted test
defender.eval(mix_x, mix_y, targeted=True, target_label=target_label)
# untargeted teat
defender.eval(mix_x, mix_y, targeted=False)

Targeted Test:
acc:0.9296440972222222, far:0.0, frr0.0
Untargeted Test:
acc:0.9298177083333333, far:0.0, frr0.0


## Anonymous 2 Model

In [ ]:
BAD_MODEL_FILE = 'anonymous_2_bd_net.h5'
CLEAN_MODEL_FILE ='repair_model/ano2_bd_repaired.h5'
bad_model = load_model(os.path.join(MODEL_DIR, BAD_MODEL_FILE))
clean_model = load_model(os.path.join(MODEL_DIR, CLEAN_MODEL_FILE))

# mix dataset with poison and clean data
POISON_DATA_FILE = 'sunglasses_poisoned_data.h5'
poison_x, poison_y = load_dataset(os.path.join(DATA_DIR, POISON_DATA_FILE))
mix_x = np.concatenate((clean_x, poison_x), axis=0)
poison_y = np.array([1283] * poison_y.shape[0])
mix_y = np.concatenate((clean_y, poison_y), axis=0)

print(mix_x.shape)
print(mix_y.shape)
target_label = np.array([4])

loading data file:/content/drive/MyDrive/ML4Cyber/data/sunglasses_poisoned_data
X_test shape (12830, 55, 47, 3)
Y_test shape (12830,)


In [ ]:
defender = BBD(bad_model=bad_model, clean_model=clean_model)
# targeted test
defender.eval(mix_x, mix_y, targeted=True, target_label=target_label)
# untargeted teat
defender.eval(mix_x, mix_y, targeted=False)

Targeted Test:
acc:0.923515625, far:0.0, frr0.0
Untargeted Test:
acc:0.9201953125, far:0.0, frr0.0


# Multi-Trigger Model

In [ ]:
BAD_MODEL_FILE = 'multi_trigger_multi_target_bd_net.h5'
CLEAN_MODEL_FILE ='repair_model/mul_bd_repaired.h5'
bad_model = load_model(os.path.join(MODEL_DIR, BAD_MODEL_FILE))
clean_model = load_model(os.path.join(MODEL_DIR, CLEAN_MODEL_FILE))

mix_x = clean_x.copy()
mix_y = clean_y.copy()
# mix dataset with poison and clean data
POISON_DATA_FILE = ['eyebrows_poisoned_data.h5', 'lipstick_poisoned_data.h5', 'sunglasses_poisoned_data.h5']
for poison_data in POISON_DATA_FILE:
    poison_x, _ = load_dataset(os.path.join(DATA_DIR, poison_data))
    poison_y = np.array([1283] * _.shape[0])
    mix_x = np.concatenate((mix_x, poison_x), axis=0)
    mix_y = np.concatenate((mix_y, poison_y), axis=0)

print('mix x:', mix_x.shape)
print('mix y', mix_y.shape)

target_label = np.array([1, 5, 8])

loading data file:/content/drive/MyDrive/ML4Cyber/data/eyebrows_poisoned_data
X_test shape (10264, 55, 47, 3)
Y_test shape (10264,)
loading data file:/content/drive/MyDrive/ML4Cyber/data/lipstick_poisoned_data
X_test shape (10264, 55, 47, 3)
Y_test shape (10264,)
loading data file:/content/drive/MyDrive/ML4Cyber/data/sunglasses_poisoned_data
X_test shape (12830, 55, 47, 3)
Y_test shape (12830,)
mix x: (46188, 55, 47, 3)
mix y (46188,)


In [ ]:
defender = BBD(bad_model=bad_model, clean_model=clean_model)
# targeted test
defender.eval(mix_x, mix_y, targeted=True, target_label=target_label)
# untargeted teat
defender.eval(mix_x, mix_y, targeted=False)

Targeted Test:
acc:0.9414441747572816, far:0.0, frr0.0
Untargeted Test:
acc:0.9427011095700416, far:0.0, frr0.0
